# Evaluations

In [10]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
# notebook path get folder path, python path gets abs path
current_dir = Path.cwd().parent
os.chdir(current_dir)

import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

print(f"Current working directory is now: {Path.cwd()}")

from preprocess.data_preprocessing import *
from features.featuring import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current working directory is now: /Users/kevinb/Desktop/cse158/RSDB


In [11]:
url = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-California_10.json.gz"
meta_url = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/meta-California.json.gz"

cleaned_df = get_clean_review_data(url, meta_url)
featured_df = featuring_engineering(cleaned_df)
data_query = featured_df[['gmap_id', 'reviewer_id', 'rating']]

/Users/kevinb/Desktop/cse158/RSDB/rsdb
Loading metadata from: /Users/kevinb/Desktop/cse158/RSDB/rsdb/data/metadata.json.gz
Loaded 463034 metadata entries.
Processing review data from: /Users/kevinb/Desktop/cse158/RSDB/rsdb/data/data.json.gz


Processing chunks: 101it [01:17,  1.31it/s, Processed Rows=9581114]                         


Processed 9581114 review entries.
finished finding generalized categories. Takes 8.154163122177124
finished bining locations. Takes 9.085221290588379
finished featuring hours. Takes 49.04840016365051
finished creating model specalizied feature. Takes 22.112358808517456


In [17]:
train_df = featured_df.sample(frac=0.8, random_state=42)
test_df = featured_df.drop(train_df.index)

In [13]:
print(f"Current working directory is now: {Path.cwd()}")
model_path = 'trained_fpmc_model'
model = tf.keras.models.load_model(model_path)
print(model.summary())

Current working directory is now: /Users/kevinb/Desktop/cse158/RSDB
Model: "fpmc_variants"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  18724680  
                                                                 
 embedding_1 (Embedding)     multiple                  3968224   
                                                                 
 embedding_2 (Embedding)     multiple                  3968224   
                                                                 
 embedding_3 (Embedding)     multiple                  180045    
                                                                 
 embedding_4 (Embedding)     multiple                  38156     
                                                                 
 embedding_5 (Embedding)     multiple                  312       
                                                   

In [18]:
from train import fpmc_df_to_tf_dataset, fpmc_df_to_tf_dataset
# train_data = tlfm_df_to_tf_dataset(train_df).shuffle(1024).batch(4096)
# test_data = tlfm_df_to_tf_dataset(test_df).batch(4096)

train_data = fpmc_df_to_tf_dataset(train_df).shuffle(1024).batch(4096)
test_data = fpmc_df_to_tf_dataset(test_df).batch(4096)

In [19]:
test_metrics = model.evaluate(test_data, return_dict=True)
print(f"Test RMSE: {test_metrics['root_mean_squared_error']}")

365/365 [==============================] - 16s 44ms/step - loss: 0.0068 - root_mean_squared_error: 0.0000e+00
Test RMSE: 0.0


# Downstream Applications

In [27]:
from recommendation import *

[autoreload of preprocess.data_preprocessing failed: Traceback (most recent call last):
  File "/Users/kevinb/miniforge3/envs/dsc80/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/Users/kevinb/miniforge3/envs/dsc80/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/Users/kevinb/miniforge3/envs/dsc80/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 839, in exec_module
  File "<frozen importlib._bootstrap_external>", line 976, in get_code
  File "<frozen importlib._bootstrap_external>", line 906, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/kevinb/Desktop/cse158/RSDB/rsdb/preprocess/data_preprocessing.py", li

In [34]:
type(model)

keras.src.saving.legacy.saved_model.load.FPMCVariants

In [30]:
cleaned_df.sample(5)['gmap_id']

7200388    0x809607d724ebf7d9:0x83f6a980a640dd5c
5229231    0x809080b0a4470831:0xea304e75365d924a
7191235     0x80c2ba68c6630487:0x2ecadfe5773afdf
5851399    0x8085873db24dd24d:0x4d95ceddc399bfc8
2547734    0x808f7d9f01fbd9c3:0xeb344f103f8fad40
Name: gmap_id, dtype: object

In [37]:
rec = Recommendation(model, cleaned_df, "FPMCVariants")

finished finding generalized categories. Takes 10.115740060806274
finished bining locations. Takes 10.386879920959473
finished featuring hours. Takes 64.5651490688324
finished creating model specalizied feature. Takes 32.98767280578613


In [41]:
rec_users = rec.recommend("0x809607d724ebf7d9:0x83f6a980a640dd5c")

In [50]:
rec_users["reviewer_id"] = rec_users["reviewer_id"].astype(float)

In [57]:
cleaned_df.merge(rec_users, on="reviewer_id").category.explode().value_counts()

Restaurant                                     54
Tourist attraction                             46
Cafe                                           28
Park                                           26
Hiking area                                    24
                                               ..
Catholic church                                 1
Church of Jesus Christ of Latter-day Saints     1
Spanish restaurant                              1
Taco restaurant                                 1
Truck accessories store                         1
Name: category, Length: 452, dtype: int64

In [60]:
merged = cleaned_df.merge(rec_users, on="reviewer_id")
merged

,reviewer_id,reviewer_name,review_time(unix),rating,text,resp,gmap_id,has_rep,gmap_name,address,...,longitude,description,category,avg_rating,num_of_reviews,price,hours,MISC,relative_results,pred_rating
0,1.171237e+20,Gyan Penrose-Kafka,1566694522636,5,Super chill space with yummy kombucha (lots in...,None,0x80dc7588a897ed91:0x190ac94f6ebb8c76,False,The Boochyard @ Local Roots,"The Boochyard @ Local Roots, 1430 Vantage Ct S...",...,-117.224863,NaN,[Brewery],4.9,38,NaN,"[[Thursday, 3–8PM], [Friday, 3–9PM], [Saturday...","{'Service options': ['Delivery'], 'Accessibili...","[0x80dc76cb9d795c1f:0x9c650e782f4ee83b, 0x80dc...",4.251787
1,1.171237e+20,Gyan Penrose-Kafka,1600788502478,4,Dance North County is conveniently located but...,None,0x80dc0c6e09f37f0b:0x45e22768c8314a8e,False,Dance North County,"Dance North County, 535 Encinitas Blvd Suite 1...",...,-117.279725,NaN,"[Dance company, Dance school]",4.8,17,NaN,"[[Wednesday, 8AM–10PM], [Thursday, 8AM–10PM], ...","{'Health & safety': ['Appointment required', '...","[0x80dc0c6e09f37f0b:0x646c7857339b6af2, 0x80dc...",4.251787
2,1.171237e+20,Gyan Penrose-Kafka,1577458451779,5,"Part of Penasquitos Canyon Preserve, Wagon Whe...",None,0x80dc07a74b4be999:0xbc176fa13d90f05b,False,Wagon Wheel Crossing,"Wagon Wheel Crossing, San Diego, CA 92121",...,-117.202955,NaN,[Bridge],4.6,18,NaN,NaN,{'Accessibility': ['Wheelchair accessible entr...,"[0x80dbf87a064c02af:0xa8fd9aad7728bbb6, 0x80dc...",4.251787
3,1.171237e+20,Gyan Penrose-Kafka,1549586075738,5,These guys did a great oil change. That means ...,"{'time': 1549589479668, 'text': 'Thank you so ...",0x80dc705097d04c1f:0x3632d855f8b8491c,True,All Tune and Lube (Total Car Care),"All Tune and Lube (Total Car Care), 3965 Missi...",...,-117.321805,NaN,"[Auto repair shop, Brake shop]",4.5,75,NaN,"[[Monday, Closed], [Tuesday, 8AM–4:30PM], [Wed...",{'Accessibility': ['Wheelchair accessible entr...,"[0x80dc7049bd7fe4e7:0x745a501eb14b63b, 0x80dc7...",4.251787
4,1.171237e+20,Gyan Penrose-Kafka,1539810029964,5,"I'm normally skeptical of chiropractors, with ...",None,0x80dc0741cb51d349:0x2cc6989cea3fb53b,False,Biophysics Chiropractic,"Biophysics Chiropractic, 5820 Oberlin Dr #204,...",...,-117.194081,NaN,"[Chiropractor, Massage therapist, Physical the...",5.0,98,NaN,"[[Sunday, Closed], [Monday, 8:30AM–1PM], [Tues...",{'Accessibility': ['Wheelchair accessible entr...,"[0x80dc00718ace3a4f:0x3a8014ae68672c33, 0x80dc...",4.251787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,1.029571e+20,Dr. Damian Corbin Jenkins,1527439764957,5,Best kept secret. There is a server there who...,None,0x80e820c3814dc591:0xdf6e3c94dfab74e1,False,Calabasas Patio Cafe,"Calabasas Patio Cafe, 26651 Agoura Rd #100, Ca...",...,-118.704322,NaN,[Restaurant],4.0,27,NaN,"[[Thursday, 7AM–3PM], [Friday, 7AM–3PM], [Satu...","{'Service options': ['Outdoor seating', 'Takeo...","[0x80e820c6ffb11959:0xba6bedb0ab458de4, 0x80e8...",4.251888
462,1.029571e+20,Dr. Damian Corbin Jenkins,1527440090133,1,Don't ever go there. Salmonella outbreak!,None,0x80e820dafb23ea67:0x2646d441f1b7f34c,False,Grape Arbor Park,"Grape Arbor Park, 5100 Parkville Rd, Calabasas...",...,-118.710562,"Small, grassy park featuring open & covered pi...","[Park, Tourist attraction]",4.5,78,NaN,"[[Wednesday, Open 24 hours], [Thursday, Open 2...",{'Accessibility': ['Wheelchair accessible entr...,"[0x80c29e3789dbc5af:0xe3c6ab34f8aef191, 0x80c2...",4.251888
463,1.029571e+20,Dr. Damian Corbin Jenkins,1527440033303,5,This year the concerts and musicals have been ...,None,0x80e826b19f5ae169:0x26cd7ef954fc5a63,False,The Performing Arts Education Center (A-PAEC),"The Performing Arts Education Center (A-PAEC),...",...,-118.752322,NaN,[Performing arts theater],4.8,53,NaN,NaN,NaN,"[0x80e826ad4a3c073d:0x26cd4b5e750a619e, 0x80e8...",4.251888
464,1.016830e+20,Christy Paton,1538002586150,5,Lots of great shops and places to stop for food.,None,0x80dc72fb37895853:0xb4f60b7ac07956f2,False,Carlsbad Premium 

In [61]:
merged[merged['gmap_id']=="0x80da9498921bae3f:0x632104cc9386eb2a"].rating

465    5
Name: rating, dtype: int64

In [67]:
# Filter the dataframe to include only rows where the category contains 'Tourist attraction'
tourist_attraction_df = cleaned_df[cleaned_df['category'].apply(lambda x: 'Tourist attraction' in x)]

# Check the ratings of the filtered dataframe
tourist_attraction_ratings = tourist_attraction_df['rating']
print(tourist_attraction_ratings.describe())
print(tourist_attraction_ratings.value_counts(normalize=True))

count    468502.000000
mean          4.523569
std           0.834121
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: rating, dtype: float64
5    0.680070
4    0.213433
3    0.073144
2    0.016702
1    0.016651
Name: rating, dtype: float64


90% of the users that matches a hiking location also give high ratings to similar places in there historical interactions

In [54]:
cleaned_df[cleaned_df["gmap_id"]=="0x809607d724ebf7d9:0x83f6a980a640dd5c"]

,reviewer_id,reviewer_name,review_time(unix),rating,text,resp,gmap_id,has_rep,gmap_name,address,latitude,longitude,description,category,avg_rating,num_of_reviews,price,hours,MISC,relative_results
7200347,1.123561e+20,CJ Brown,1619632187447,4,Great easy trail. Currently completely melted ...,None,0x809607d724ebf7d9:0x83f6a980a640dd5c,False,Fisherman's Trail—Convict Lake,"Fisherman's Trail—Convict Lake, Fisherman's Tr...",37.588651,-118.854691,NaN,[Hiking area],4.7,68,NaN,NaN,{'Amenities': ['Good for kids']},"[0x8096098a8a7a2bb9:0x929b269e36d3e082, 0x8096..."
7200348,1.177597e+20,Jesus Zamora garcia,1621178334186,5,Love it its awsome,None,0x809607d724ebf7d9:0x83f6a980a640dd5c,False,Fisherman's Trail—Convict Lake,"Fisherman's Trail—Convict Lake, Fisherman's Tr...",37.588651,-118.854691,NaN,[Hiking area],4.7,68,NaN,NaN,{'Amenities': ['Good for kids']},"[0x8096098a8a7a2bb9:0x929b269e36d3e082, 0x8096..."
7200349,1.070637e+20,Vernon Yue,1621098404364,5,Easy hike going around the lake .,None,0x809607d724ebf7d9:0x83f6a980a640dd5c,False,Fisherman's Trail—Convict Lake,"Fisherman's Trail—Convict Lake, Fisherman's Tr...",37.588651,-118.854691,NaN,[Hiking area],4.7,68,NaN,NaN,{'Amenities': ['Good for kids']},"[0x8096098a8a7a2bb9:0x929b269e36d3e082, 0x8096..."
7200350,1.130269e+20,jim witte,1523969581788,5,Convict Lake is a beautiful spot at the base o...,None,0x809607d724ebf7d9:0x83f6a980a640dd5c,False,Fisherman's Trail—Convict Lake,"Fisherman's Trail—Convict Lake, Fisherman's Tr...",37.588651,-118.854691,NaN,[Hiking area],4.7,68,NaN,NaN,{'Amenities': ['Good for kids']},"[0x8096098a8a7a2bb9:0x929b269e36d3e082, 0x8096..."
7200351,1.181755e+20,Christian Tapley,1612116910060,5,This place is so pretty I love it,None,0x809607d724ebf7d9:0x83f6a980a640dd5c,False,Fisherman's Trail—Convict Lake,"Fisherman's Trail—Convict Lake, Fisherman's Tr...",37.588651,-118.854691,NaN,[Hiking area],4.7,68,NaN,NaN,{'Amenities': ['Good for kids']},"[0x8096098a8a7a2bb9:0x929b269e36d3e082, 0x8096..."
7200352,1.100460e+20,Honey Alabado,1611974344936,5,The view is overwhelming. a nice place to go f...,None,0x809607d724ebf7d9:0x83f6a980a640dd5c,False,Fisherman's Trail—Convict Lake,"Fisherman's Trail—Convict Lake, Fisherman's Tr...",37.588651,-118.854691,NaN,[Hiking area],4.7,68,NaN,NaN,{'Amenities': ['Good for kids']},"[0x8096098a8a7a2bb9:0x929b269e36d3e082, 0x8096..."
7200353,1.016656e+20,Anahi Rincon,1576002536375,5,It’s one of the most beautiful places that I h...,None,0x809607d724ebf7d9:0x83f6a980a640dd5c,False,Fisherman's Trail—Convict Lake,"Fisherman's Trail—Convict Lake, Fisherman's Tr...",37.588651,-118.854691,NaN,[Hiking area],4.7,68,NaN,NaN,{'Amenities': ['Good for kids']},"[0x8096098a8a7a2bb9:0x929b269e36d3e082, 0x8096..."
7200354,1.125205e+20,Joshua Kim,1567004834197,5,Very well kept trail around Convict Lake. Seve...,None,0x809607d724ebf7d9:0x83f6a980a640dd5c,False,Fisherman's Trail—Convict Lake,"Fisherman's Trail—Convict Lake, Fisherman's Tr...",37.588651,-118.854691,NaN,[Hiking area],4.7,68,NaN,NaN,{'Amenities': ['Good for kids']},"[0x8096098a8a7a2bb9:0x929b269e36d3e082, 0x8096..."
7200355,1.031773e+20,Jansen Wendlandt,1563404275489,4,Nicely maintained loop around Convict Lake.\n\...,None,0x809607d724ebf7d9:0x83f6a980a640dd5c,False,Fisherman's Trail—Convict Lake,"Fisherman's Trail—Convict Lake, Fisherman's Tr...",37.588651,-118.854691,NaN,[Hiking area],4.7,68,NaN,NaN,{'Amenities': ['Good for kids']},"[0x8096098a8a7a2bb9:0x929b269e36d3e082, 0x8096..."
7200356,1.073675e+20,DANNY KidSuper LIN,1566018920152,5,Awesome place made most memorable with the you...,None,0x809607d724ebf7d9:0x83f6a980a640dd5c,False,Fisherman's Trail—Convict Lake,"Fisherman's Trail—Convict Lake, Fisherman's Tr...",37.588651,-118.854691,NaN,[Hiking area],4.7,68,NaN,NaN,{'Amenities': ['Good for kids']},"[0x8096098a8a7a2bb9:0x929b269e36d3e082, 0x8096..."
